### Conversation Summary Memory 
الكود بيبني نظام محادثة بين إنسان وذكاء اصطناعي، الذكاء الصناعي بيحتفظ `بملخص` المحادثة باستخدام ConversationSummaryMemory علشان يتذكر الحوار، بدل ما يخزن كل الكلام كامل. وكمان بيستخدم LLMChain عشان يربط الـ model بالـ prompt والذاكرة مع بعض

In [49]:
import openai
from openai import OpenAI
import langchain
import os
# Step 1: Load environment variables
%reload_ext dotenv
%dotenv
# Step 3: Access your OpenAI API key from environment variables
for key, value in os.environ.items():
    if key == "OPENAI_API_KEY":
        print(f"{key}: value")

OPENAI_API_KEY: value


In [50]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [51]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(model='gpt-4o-mini', temperature=0,max_completion_tokens=50)
# response = chat.invoke("I have recently adopted a dog. Could you suggest some dog names?")
# print(response.content)

In [52]:
from langchain_openai.chat_models import ChatOpenAI

from langchain_core.messages import SystemMessage
from langchain_core.prompts import (ChatPromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    MessagesPlaceholder,
                                    PromptTemplate)

from langchain.chains.llm import LLMChain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory,ConversationSummaryMemory

from langchain.globals import set_verbose
set_verbose(True)

In [53]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI.  
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human:
{question}

AI:
'''

الـ message_log هو عبارة عن ملخص للمحادثة اللي حصلت بين الإنسان والذكاء الصناعي.
الـ ConversationSummaryMemory بيبني الملخص ده بعد كل سؤال وجواب، وبيخزنه عشان لما تيجي تبعت سؤال جديد، يتحط جوه {message_log} في الـ TEMPLAT

In [54]:
prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [55]:
chat_memory = ConversationSummaryMemory(llm=chat,
                                        memory_key='message_log',
                                        return_messages=False)
chat_memory

ConversationSummaryMemory(llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002859567D000>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002859567FD60>, root_client=<openai.OpenAI object at 0x000002859566EF80>, root_async_client=<openai.AsyncOpenAI object at 0x000002859567D060>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=50), chat_memory=InMemoryChatMessageHistory(messages=[]), memory_key='message_log')

In [56]:
chat_memory.load_memory_variables({})

{'message_log': ''}

In [57]:
chain = LLMChain(llm=chat,prompt=prompt_template, memory=chat_memory)

In [58]:
response =  chain.invoke({'question':"Can you give me an interesting fact I probably didn't know about?"})
response



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI.  
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:


Human:
Can you give me an interesting fact I probably didn't know about?

AI:



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their f

{'question': "Can you give me an interesting fact I probably didn't know about?",
 'message_log': '',
 'text': 'Sure! Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible!'}

In [59]:
print(response['text'])

Sure! Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible!


In [60]:
response =  chain.invoke({'question':"Can you talk about rodri?"})
response



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI.  
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human asks the AI for an interesting fact. The AI shares that honey never spoils, citing that archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible.

Human:
Can you talk about rodri?

AI:



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full poten

{'question': 'Can you talk about rodri?',
 'message_log': 'The human asks the AI for an interesting fact. The AI shares that honey never spoils, citing that archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible.',
 'text': 'Sure! Rodri, whose full name is Rodrigo Hernández Cascante, is a Spanish professional footballer who plays as a defensive midfielder. He is known for his strong tackling, passing ability, and tactical awareness. Rodri began his career at Villarreal'}

In [63]:
response =  chain.invoke({'question':"Can you talk about Kevin debroyne?"})
response



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI.  
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human asks the AI for an interesting fact, and the AI shares that honey never spoils, noting that archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. The human

Human:
Can you talk about Kevin debroyne?

AI:



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans

{'question': 'Can you talk about Kevin debroyne?',
 'message_log': 'The human asks the AI for an interesting fact, and the AI shares that honey never spoils, noting that archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. The human',
 'text': 'Kevin De Bruyne is a Belgian professional footballer who plays as a midfielder for Manchester City in the English Premier League and the Belgium national team. Known for his exceptional passing, vision, and ability to score goals, De Bruyne is often regarded as'}

In [64]:
chat_memory.load_memory_variables({})

{'message_log': 'The human asks the AI for an interesting fact, and the AI shares that honey never spoils, noting that archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. The human'}